In [1]:
# Import many dataFrame for the Algorithm Comparison:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.lines import Line2D
from scipy import stats
import glob
import os
# pd.set_option('display.max_rows', 500)
from scipy.stats import gaussian_kde
from scipy.stats import sem
import seaborn as sns
from scipy.stats.mstats import winsorize

### Import our libraries
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../data_preprocessing/')
import fixation_plots as plots
from utility import utilitiesCalc
import interpolationET as inter
counter = 22
lb_data = []
el_data = []
trial_data = []
targets_data = []
subject = 0
targets_data = []
el_blinks = pd.DataFrame()
el_blnk = []
s_gaze = 0
all_target = pd.DataFrame()
temp_all_data = []
df_blinks_el = pd.DataFrame()
perc_loss_el_arr = []
perc_loss_lb_arr = []

for i in range(1,counter):
    subject = subject + 1
    s_gaze = s_gaze + 1
    el_gaze = pd.read_csv("../data/el_data/p" + str(i) + '.csv', engine='python')
    el_blinks = pd.read_csv("../data/el_data/el_events/blinks/p" + str(i) + '_events.csv', engine='python')
    lb_gaze = pd.read_csv("../data/lb_data/timeseries_data/p" + str(i) + '_XYTC.csv', engine='python')
    print('loads files from =' + str(i))
    
    lb_gaze = utilitiesCalc.addParticipantNumberCol(subject, lb_gaze)
    lb_gaze = inter.formating_labvanced (lb_gaze)
    lb_gaze = lb_gaze[lb_gaze['Task_Name'] == "free_view"]
    starting_time = lb_gaze.time_lb.values[0]
    finishing_time = lb_gaze.time_lb.values[-1]
    
    lb_under_threshold = lb_gaze[lb_gaze['c'] <= 0.19579889650805132]
    
    
    
    lb_gaze_data_all = len(lb_gaze)
    perc_loss_lb = len(lb_under_threshold)/lb_gaze_data_all * 100
    print('Labvanced: for subject = ' +str(i) + 'data loss in % =' + str(perc_loss_lb))
    perc_loss_lb_arr.append(perc_loss_lb)
    mean_data_loss_lb = np.mean(perc_loss_lb_arr)
    std_data_loss_lb = np.std(perc_loss_lb_arr)
    
    # Now we can drop the "under threshold data" from gaze data
    lb_gaze.drop(lb_under_threshold.index, inplace=True)
    
    el_gaze = utilitiesCalc.addParticipantNumberCol(subject, el_gaze)
    el_gaze = el_gaze.loc[(el_gaze.Time <= finishing_time) & (el_gaze.Time >= starting_time)]
    
    el_blinks = el_blinks.loc[(el_blinks.End <= finishing_time) & (el_blinks.Start >= starting_time)]
    el_blinks = utilitiesCalc.addParticipantNumberCol(s_gaze, el_blinks)
        
    blinks = []
    df_blinks = pd.DataFrame()
    for index, row in el_blinks.iterrows():
        blink_offset = row['End']
        blink_onset = row['Start']
        within_blinks = el_gaze.loc[(el_gaze.Time <= blink_offset) & (el_gaze.Time >= blink_onset)]
        blinks.append(within_blinks)
    df_blinks_el = pd.concat(blinks, axis=0, ignore_index=False)
    el_gaze_data_all = len(el_gaze)
    perc_loss_el = len(df_blinks_el)/el_gaze_data_all * 100
    print('Eyelink: for subject = ' +str(i) + 'data loss in % =' + str(perc_loss_el))
    perc_loss_el_arr.append(perc_loss_el)
    mean_data_loss_el = np.mean(perc_loss_el_arr)
    std_data_loss_el = np.std(perc_loss_el_arr)
    el_gaze.drop(df_blinks_el.index, inplace=True)
    
    el = el_gaze.rename(columns={'X':'X_el','Y':'Y_el','Time':'t'})
    lb = lb_gaze.set_index('time_lb')
        
    el = el.set_index('t')
    
    def interpolation ():
    # Interpolate data

        df_temp = pd.concat([el, lb .index.to_frame()]).sort_index().interpolate()

        df_temp = df_temp[~df_temp.index.duplicated(keep='first')]
        df_interpolated = lb .merge(df_temp, left_index=True, right_index=True, how='left')

        df_interpolated = df_interpolated.drop(columns=['time_lb'])
        df_interpolated = df_interpolated.reset_index()

        return df_interpolated
    df_interpolated = interpolation()
    df_interpolated.reset_index(inplace=True)
    
    temp_all_data.append(df_interpolated)
    df = pd.concat(temp_all_data, axis=0, ignore_index=True)
    
df.to_csv('../data/free_view_interpolated/free_view_inter.csv', index = False)
# df.to_csv('../data/head_interpolated/head_inter.csv', index = False)
print('Labvanced Mean data lost over all participants M=' + str(mean_data_loss_lb))
print('Eyelink Mean data lost over all participants M=' + str(mean_data_loss_el))

loads files from =1
Labvanced: for subject = 1data loss in % =0.4413619167717529
Eyelink: for subject = 1data loss in % =1.3449135150982117
loads files from =2
Labvanced: for subject = 2data loss in % =1.1313639220615965
Eyelink: for subject = 2data loss in % =1.5762061403508771
loads files from =3
Labvanced: for subject = 3data loss in % =11.132194813409235
Eyelink: for subject = 3data loss in % =2.1011919150597818
loads files from =4
Labvanced: for subject = 4data loss in % =2.9320024953212727
Eyelink: for subject = 4data loss in % =3.9476149826146703
loads files from =5
Labvanced: for subject = 5data loss in % =2.0012507817385865
Eyelink: for subject = 5data loss in % =1.3316758019249968
loads files from =6
Labvanced: for subject = 6data loss in % =0.12578616352201258
Eyelink: for subject = 6data loss in % =4.497957185623106
loads files from =7
Labvanced: for subject = 7data loss in % =0.7471980074719801
Eyelink: for subject = 7data loss in % =5.076010945576162
loads files from =8
L

In [2]:
df_blinks_el

,X,Y,Tracker_Time,Time,Participant_Nr
801915,0.0,0.0,2784251,1.657092e+12,21
801916,0.0,0.0,2784253,1.657092e+12,21
801917,0.0,0.0,2784255,1.657092e+12,21
801918,0.0,0.0,2784257,1.657092e+12,21
801919,0.0,0.0,2784259,1.657092e+12,21
...,...,...,...,...,...
850893,0.0,0.0,2882207,1.657092e+12,21
850894,0.0,0.0,2882209,1.657092e+12,21
850895,0.0,0.0,2882211,1.657092e+12,21
850896,0.0,0.0,2882213,1.657092e+12,21


In [3]:
lb_under_threshold

,Block_Nr,Block_Name,Task_Nr,Task_Name,Trial_Nr,Trial_Id,Exp_Subject_Id,Rec_Session_Id,Session_Nr,timestamp,X_lb,Y_lb,time_lb,c,Participant_Nr
9964,1,block_1,7,free_view,3,5,452085,515707,1,1657091656355,828.271937,227.015948,1657091656326,0.00000,21
9965,1,block_1,7,free_view,3,5,452085,515707,1,1657091656367,956.531310,329.119022,1657091656359,0.00000,21
9966,1,block_1,7,free_view,3,5,452085,515707,1,1657091656421,1166.459527,466.031091,1657091656392,0.00000,21
9967,1,block_1,7,free_view,3,5,452085,515707,1,1657091656433,1191.924334,301.772051,1657091656425,0.00000,21
9968,1,block_1,7,free_view,3,5,452085,515707,1,1657091656485,973.487806,75.947928,1657091656458,0.06393,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11306,1,block_1,7,free_view,18,10,452085,515707,1,1657091746375,803.734918,205.221793,1657091746359,0.00000,21
11307,1,block_1,7,free_view,18,10,452085,515707,1,1657091746415,890.029221,284.737583,1657091746392,0.00000,21
11308,1,block_1,7,free_view,18,10,452085,515707,1,1657091746437,955.649443,350.327423,1657091746425,0.00000,21
11309,1,block_1,7,free_view,18,10,452085,515707,1,1657091746485,1004.715929,297.594336,1657091746458,0.00000,21
